In [1]:
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import openml
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *

/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X = sample_real_data_X(source="csv",file_path= "/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data/CCLE/X_ccle_rnaseq_Topotecan_top500.csv",sample_row_n= None)

In [3]:
y = sample_real_data_y(source="csv", file_path="/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data/CCLE/y_ccle_rnaseq_Topotecan.csv")[0]

In [4]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_train = X_train[:,:5]

In [6]:
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)
rf_plus_base = RandomForestPlusRegressor(rf_model=est)
rf_plus_base.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.7s finished


In [7]:
if X_train.shape[0] > 100:
    indices_train = np.random.choice(X_train.shape[0], 100, replace=False)
    X_train_subset = X_train[indices_train]
    y_train_subset = y_train[indices_train]
else:
    indices_train = np.arange(X_train.shape[0])
    X_train_subset = X_train
    y_train_subset = y_train

if X_test.shape[0] > 100:
    indices_test = np.random.choice(X_test.shape[0], 100, replace=False)
    X_test_subset = X_test[indices_test]
    y_test_subset = y_test[indices_test]
else:
    indices_test = np.arange(X_test.shape[0])
    X_test_subset = X_test
    y_test_subset = y_test

In [8]:
rf_plus_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")

In [9]:
rf_plus_mdi.explain(X=X_train, y=y_train)

(array([[-0.93740875, -1.09132836, -1.35028366, -1.15388852, -1.25685594],
        [-1.04895378, -1.34183556, -1.15525384, -1.32002817, -1.36718094],
        [-0.67151686, -0.2809473 , -0.63317469, -0.341039  , -0.36653009],
        ...,
        [-1.13306773, -1.41037856, -1.23963396, -1.42238344, -1.39783221],
        [-1.71587687, -1.58479507, -1.81213149, -1.73044741, -1.69754925],
        [-0.43752179, -0.4671859 , -0.49108445, -0.47976167, -0.46289299]]),
 array([[3.27029125, 3.11637164, 2.85741634, 3.05381148, 2.95084406],
        [2.64835378, 2.94123556, 2.75465384, 2.91942817, 2.96658094],
        [3.30621686, 2.89714089, 3.26787469, 2.97383506, 3.00123009],
        ...,
        [2.70416773, 2.98147856, 2.81073396, 2.99348344, 2.96893221],
        [2.97227687, 2.84119507, 3.06853149, 2.98684741, 2.95394925],
        [2.93105958, 2.9635859 , 2.98748445, 2.97616167, 2.95547056]]))

In [19]:
temp = rf_plus_mdi.explain(X=X_train, y=y_train)[0][:5]

In [24]:
temp = np.abs(temp)
temp

array([[0.93740875, 1.09132836, 1.35028366, 1.15388852, 1.25685594],
       [1.04895378, 1.34183556, 1.15525384, 1.32002817, 1.36718094],
       [0.67151686, 0.2809473 , 0.63317469, 0.341039  , 0.36653009],
       [0.67514285, 0.64116806, 0.33789905, 0.58319897, 0.51105166],
       [0.48467372, 0.26272469, 0.30583033, 0.36102418, 0.39570995]])

In [25]:
np.argsort(-1*temp)

array([[2, 4, 3, 1, 0],
       [4, 1, 3, 2, 0],
       [0, 2, 4, 3, 1],
       [0, 1, 3, 4, 2],
       [0, 4, 3, 2, 1]])

In [26]:
np.mean(np.argsort(-1*temp), axis=0)

array([1.2, 2.4, 3.2, 2.4, 0.8])

In [13]:
rf_plus_mdi.explain(X=X_train, y=y_train)[1][0]

array([3.27029125, 3.11637164, 2.85741634, 3.05381148, 2.95084406])

In [17]:
rf_plus_base.estimators_

AttributeError: 'list' object has no attribute 'estimators_'

In [15]:
y_train[0]

4.2077

In [ ]:
treeshap_fi, _, _, _ = tree_shap_evaluation_RF(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset, fit=est, mode="absolute", train_only=True)

In [ ]:
lmdi_fi, _, _, _ = LFI_evaluation_RFPlus_oob_l2_norm_sign(X_train, y_train, X_train_subset, y_train_subset, X_test, y_test, X_test_subset, y_test_subset,  fit=rf_plus_base, mode="absolute", train_only=True)

In [ ]:
def select_top_features(array, sorted_indices, percentage):
    array = copy.deepcopy(array)
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * percentage))
    selected_indices = sorted_indices[:num_selected]
    selected_array = array[:, selected_indices]
    return num_selected, selected_array

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_shap_mse = []
metric_results_shap_r2 = []
train_fi_mean = np.mean(treeshap_fi, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)}
                    #"Linear": LinearRegression(),
                    #"XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    #"RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    # for a_model in ablation_models:
    #     ablation_models[a_model].fit(X_train_masked, y_train)
    rf = LinearRegression()# RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42)
    rf.fit(X_train_masked, y_train)
    y_pred = rf.predict(X_test_masked)
    metric_results_shap_mse.append(mean_squared_error(y_test, y_pred))
    metric_results_shap_r2.append(r2_score(y_test, y_pred))

In [ ]:
select_top_features(X_train, sorted_feature, 0.01)[1][0]

In [ ]:
select_top_features(X_test, sorted_feature, 0.01)[1][0]

In [ ]:
indices = np.nonzero(np.isin(X_train[0], select_top_features(X_train, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
indices = np.nonzero(np.isin(X_test[0], select_top_features(X_test, sorted_feature, 0.01)[1][0]))[0]

print(indices)

In [ ]:
X_train[0]

In [ ]:
metric_results_shap_mse

In [ ]:
# plot metric_results_shap_r2
metric_results_shap_r2 = np.array(metric_results_shap_r2).reshape(len(mask_ratio), -1)
plt.figure()
plt.plot(mask_ratio, metric_results_shap_r2[:, 0], label="RF_Regressor")
plt.xlabel("Feature Ratio")
plt.ylabel("R2")
plt.legend()
plt.show()

In [ ]:
mask_ratio = [0.05, 0.1, 0.25, 0.5, 0.9]
metric_results_lmdi_mse = []
metric_results_lmdi_r2 = []
train_fi_mean = np.mean(local_fi_score_train, axis=0)
sorted_feature = np.argsort(-train_fi_mean)
for mask in mask_ratio:
    print(X_train.shape)
    num_features_masked, X_train_masked = select_top_features(X_train, sorted_feature, mask)
    print(X_train_masked.shape)
    num_features_masked, X_test_masked = select_top_features(X_test, sorted_feature, mask)
    print(X_test_masked.shape)
    ablation_models = {"RF_Regressor": RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42),
                    "Linear": LinearRegression(),
                    "XGB_Regressor": xgb.XGBRegressor(random_state=42),
                    # 'Kernel_Ridge': KernelRidge(),
                    "RF_Plus_Regressor": RandomForestPlusRegressor(rf_model=RandomForestRegressor(n_estimators=100,min_samples_leaf=5,max_features=0.33,random_state=42))}
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict(X_test_masked)
        metric_results_lmdi_mse.append(mean_squared_error(y_test, y_pred))
        metric_results_lmdi_r2.append(r2_score(y_test, y_pred))